In [ ]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
import xgboost as xgb

def monitor_model_performance(model_registry_path, test_data_path):
    """
    Monitor performance difference between current and previous model.
    
    Acceptable ranges:
    - CRITICAL: MAPE increase > 1.0 percentage point → ROLLBACK
    - WARNING: MAPE increase > 15% relative → INVESTIGATE  
    - PASS: All other cases
    
    Returns: status ('PASS', 'WARNING', 'FAIL')
    """
    
    # Define thresholds
    ABSOLUTE_THRESHOLD = 1.0  # percentage points
    RELATIVE_THRESHOLD = 15   # percent
    
    # Helper functions
    def load_model(model_dir):
        model = xgb.XGBRegressor()
        model.load_model(str(model_dir / "model.json"))
        return model
    
    def load_metadata(model_dir):
        with open(model_dir / "metadata.json", 'r') as f:
            return json.load(f)
    
    def calculate_mape(actual, predicted):
        return np.median(np.abs(actual - predicted) / actual) * 100
    
    def inv_transform(y, mean, std):
        return np.exp((y * std) + mean)
    
    # Get current and previous models
    registry = Path(model_registry_path)
    all_models = sorted([d for d in registry.iterdir() if d.is_dir() and d.name.startswith('model_')])
    
    if len(all_models) < 2:
        print("Not enough models for comparison")
        return 'BASELINE'
    
    current_model_dir = all_models[-1]
    previous_model_dir = all_models[-2]
    
    # Load test data
    test_df = pd.read_csv(f'{test_data_path}/test.csv', index_col=0)
    
    # Evaluate both models
    results = {}
    for name, model_dir in [('current', current_model_dir), ('previous', previous_model_dir)]:
        model = load_model(model_dir)
        metadata = load_metadata(model_dir)
        
        features = metadata['features']
        X_test = test_df[features]
        
        y_pred = model.predict(X_test)
        predicted_prices = inv_transform(y_pred, metadata['train_mean'], metadata['train_std'])
        actual_prices = test_df['price'].values
        
        mape = calculate_mape(actual_prices, predicted_prices)
        results[name] = {'mape': mape, 'model': model_dir.name}
    
    # Compare performance
    mape_diff = results['current']['mape'] - results['previous']['mape']
    mape_pct_change = (mape_diff / results['previous']['mape']) * 100
    
    # Determine status
    if mape_diff > ABSOLUTE_THRESHOLD:
        status = 'FAIL'
        action = 'ROLLBACK'
    elif mape_pct_change > RELATIVE_THRESHOLD:
        status = 'WARNING'
        action = 'INVESTIGATE'
    else:
        status = 'PASS'
        action = 'DEPLOY'
    
    # Print report
    print("\n" + "="*60)
    print("MODEL PERFORMANCE MONITORING")
    print("="*60)
    print(f"Current Model:  {results['current']['model']}")
    print(f"  MAPE: {results['current']['mape']:.2f}%")
    print(f"\nPrevious Model: {results['previous']['model']}")
    print(f"  MAPE: {results['previous']['mape']:.2f}%")
    print(f"\nChange: {mape_diff:+.2f} points ({mape_pct_change:+.1f}%)")
    print(f"\nStatus: {status}")
    print(f"Action: {action}")
    print("="*60 + "\n")
    
    # Save results
    monitoring_log = {
        'current_mape': results['current']['mape'],
        'previous_mape': results['previous']['mape'],
        'mape_diff': mape_diff,
        'mape_pct_change': mape_pct_change,
        'status': status,
        'action': action
    }
    
    with open(current_model_dir / "monitoring_result.json", 'w') as f:
        json.dump(monitoring_log, f, indent=2)
    
    return status

if __name__ == "__main__":
    status = monitor_model_performance(
        model_registry_path="./model_registry",
        test_data_path="./data"
    )
    
    if status == 'FAIL':
        raise ValueError("Model performance degraded - rollback recommended")
```
